In [ ]:
#@markdown ## 구글드라이브 연동
#@markdown **생성한 이미지를 구글드라이브에 자동저장하려면 체크해주세요**
Google_Drive = False #@param {type:"boolean"}

#@markdown **초기모델(checkpoint)을 구글드라이브에 저장하려면 체크해주세요**
#@markdown <div><font color="red">모델 파일사이즈(2GB ~ 7GB)가 크니 구글드라이브 용량을 확인해주세요</div>
#@markdown <div><font color="red">대신 한번받으면 다음 실행부터는 빨라져요</div>
Checkpoint_Google_Save = False #@param {type:"boolean"}

#@markdown -----
#@markdown *아래는 설정할 필요 없어요*

#@markdown **구글드라이브에 연결 폴더**
Google_Drive_Dir = 'webui' #@param {type:"string"}

#@markdown **폴더 설명** : 미리 만드셔도 되고 없으면 자동으로 생성해요
#@markdown * webui/output : 생성된 이미지들이 저장되요
#@markdown * webui/checkpoint : 모델(checkpoint)를 넣어주면 읽어올수 있어요
#@markdown * webui/lora : 로라(LoRA)를 넣어주면 읽어올수 있어요
#@markdown * webui/embedding : 임베딩(Textual Inversion)를 넣어주면 읽어올수 있어요
#@markdown * webui/hyperwork : 하이퍼워크(Hyperworks)를 넣어주면 읽어올수 있어요
#@markdown * webui/wildcards : 와일드카드(Wildcards)를 넣어주면 읽어올수 있어요

#@markdown **초기모델(checkpoint) URL**
Checkpoint_Default = 'https://civitai.com/api/download/models/30528' #@param {type:"string"}
#@markdown **초기모델 파일명**
Checkpoint_DefaultName = 'yorrrlmix_v21.safetensors' #@param {type:"string"}
#@markdown **Ngrok**
Ngrok_Key = '' #@param {type:"string"}

#@markdown -----
#@markdown *실행속도*

#@markdown **ControlNet을 사용하지 않으면 체크를 해제해주세요**
ControlNet = True #@param {type:"boolean"}

#@markdown **Dectect Detailer(ddetailer)를 사용하지 않으면 체크를 해제해주세요**
DDetailer = True #@param {type:"boolean"}

#@markdown **Segment Anything을 사용하지 않으면 체크를 해제해주세요**
SegmentAnything = False #@param {type:"boolean"}

#@markdown **ControlNet1.0(old) 모델을 받으려면 체크해주세요**
ControlNet1_0 = False #@param {type:"boolean"}

if Google_Drive:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/output
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/lora
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/embedding
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork
  !mkdir /content/drive/MyDrive/{Google_Drive_Dir}/wildcards

checkpoint_url = Checkpoint_Default
checkpoint_name = Checkpoint_DefaultName
checkpoint_down_path = '/content/sd-webui/models/Stable-diffusion'
if Google_Drive and Checkpoint_Google_Save:
  checkpoint_down_path = '/content/sd-webui/models/Stable-diffusion/google'

checkpoint_file = checkpoint_down_path+'/'+checkpoint_name

# https://github.com/camenduru 참고

%cd /content

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!apt -y install -qq aria2
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117

if DDetailer:
  !pip install -U openmim
  !mim install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html
  !pip install mmdet==2.28.2

!git clone -b stable1 https://github.com/neuralninja22/sd-webui
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/sd-webui/models/ESRGAN -o 4x-UltraSharp.pth
if ControlNet:
  !git clone https://github.com/Mikubill/sd-webui-controlnet /content/sd-webui/extensions/sd-webui-controlnet
if DDetailer:
  !git clone https://github.com/ninjaneural/ddetailer /content/sd-webui/extensions/ddetailer
if SegmentAnything:
  !git clone https://github.com/ninjaneural/sd-webui-segment-anything /content/sd-webui/extensions/sd-webui-segment-anything
!git clone https://github.com/camenduru/sd-webui-tunnels /content/sd-webui/extensions/sd-webui-tunnels
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui /content/sd-webui/extensions/deforum-for-automatic1111-webui
!git clone -b stable1 https://github.com/neuralninja22/sd-webui-images-browser /content/sd-webui/extensions/sd-webui-images-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/sd-webui/extensions/sd-webui-additional-networks
!git clone https://github.com/fkunn1326/openpose-editor /content/sd-webui/extensions/openpose-editor
!git clone https://github.com/jexom/sd-webui-depth-lib /content/sd-webui/extensions/sd-webui-depth-lib
!git clone https://github.com/hnmr293/posex /content/sd-webui/extensions/posex
!git clone https://github.com/pkuliyi2015/multidiffusion-upscaler-for-automatic1111 /content/sd-webui/extensions/multidiffusion-upscaler-for-automatic1111
!git clone https://github.com/adieyal/sd-dynamic-prompts /content/sd-webui/extensions/sd-dynamic-prompts
!git clone https://github.com/mcmonkeyprojects/sd-dynamic-thresholding /content/sd-webui/extensions/sd-dynamic-thresholding
!git clone https://github.com/neuralninja22/sd-webui-wd14-tagger /content/sd-webui/extensions/sd-webui-wd14-tagger
!git clone https://github.com/hnmr293/sd-webui-llul /content/sd-webui/extensions/sd-webui-llul
!git clone https://github.com/fishslot/video_loopback_for_webui /content/sd-webui/extensions/video_loopback_for_webui
!git clone https://github.com/Scholar01/sd-webui-mov2mov /content/sd-webui/extensions/sd-webui-mov2mov
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete /content/sd-webui/extensions/a1111-sd-webui-tagcomplete
!git clone https://github.com/ninjaneural/sd-webui-video-util /content/sd-webui/extensions/sd-webui-video-util
%cd /content/sd-webui
!git reset --hard

!wget https://raw.githubusercontent.com/ninjaneural/webui/master/misc/config.json -O /content/sd-webui/config.json

# 구글드라이브 연결
if Google_Drive:
  !mkdir /content/sd-webui/models/Lora
  !mkdir /content/sd-webui/models/hypernetworks
  !rm /content/sd-webui/embeddings/google
  !rm /content/sd-webui/models/Stable-diffusion/google
  !rm /content/sd-webui/models/Lora/google
  !rm /content/sd-webui/models/hypernetworks/google
  !rm /content/sd-webui/outputs
  !mv /content/sd-webui/outputs /content/sd-webui/outputs_backup
  !rm /content/sd-webui/extensions/sd-dynamic-prompts/wildcards
  !mv /content/sd-webui/extensions/sd-dynamic-prompts/wildcards /content/sd-webui/extensions/sd-dynamic-prompts/wildcards_backup
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/embedding /content/sd-webui/embeddings/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/checkpoint /content/sd-webui/models/Stable-diffusion/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/lora /content/sd-webui/models/Lora/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/hypernetwork /content/sd-webui/models/hypernetworks/google
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/output /content/sd-webui/outputs
  !ln -s /content/drive/MyDrive/{Google_Drive_Dir}/wildcards /content/sd-webui/extensions/sd-dynamic-prompts/wildcards
else:
  !rm /content/sd-webui/outputs

# controlnet models
if ControlNet and ControlNet1_0:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_canny-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_depth-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_hed-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_mlsd-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_normal-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_openpose-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_scribble-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_seg-fp16.safetensors

if ControlNet:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors

  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/openpose -o hand_pose_model.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/openpose -o body_pose_model.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/midas -o dpt_hybrid-midas-501f0c75.pt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_large_512_fp32.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/mlsd -o mlsd_tiny_512_fp32.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/hed -o network-bsds500.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth -d /content/sd-webui/extensions/sd-webui-controlnet/annotator/uniformer -o upernet_global_small.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o diff_control_sd15_temporalnet_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny-fp16.safetensors -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny-fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth -d /content/sd-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth

# sam 
if SegmentAnything:
  !mkdir /content/sd-webui/models/sam
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -d /content/sd-webui/models/sam -o sam_vit_h_4b8939.pth

# checkpoint
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{checkpoint_url}" -d {checkpoint_down_path} -o {checkpoint_name}

# vae
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {checkpoint_down_path} -o vae-ft-mse-840000-ema-pruned.vae.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d /content/sd-webui/models/VAE -o kl-f8-anime2.ckpt

# lora
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/amornlnw7/koreanDollLikeness_v15/resolve/main/koreanDollLikeness_v15.safetensors -d /content/sd-webui/models/Lora -o koreanDollLikeness_v15.safetensors

# embeddings
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://moritz.pm/files/ulzzang-6500.pt -d /content/sd-webui/embeddings -o ulzzang-6500.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.pt -d /content/sd-webui/embeddings -o EasyNegative.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/AsciiP/badhandv4/resolve/main/badhandv4.pt -d /content/sd-webui/embeddings -o badhandv4.pt

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/sd-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/sd-webui/launch.py

!mkdir /content/sd-webui/extensions/deforum-for-automatic1111-webui/models

if Ngrok_Key:
  !python launch.py --xformers --theme dark --ngrok {Ngrok_Key}
else:
  !python launch.py --share --xformers --enable-insecure-extension-access --theme dark --gradio-queue --cloudflared
